In [1]:
from riotwatcher import LolWatcher
import pandas as pd
import time
from tqdm import tqdm

In [38]:
import pandas as pd
from collections import Counter

# Load data
df = pd.read_csv('match_drafts.csv')

print(f"Total matches: {len(df)}")
print(f"Blue team wins: {df['blue_win'].sum()} ({df['blue_win'].mean()*100:.1f}%)")
print(f"Red team wins: {df['red_win'].sum()} ({df['red_win'].mean()*100:.1f}%)")

print("\n" + "=" * 50)
print("MOST PICKED CHAMPIONS")
print("=" * 50)

# Parse champion lists (they're saved as strings)
import ast

all_picks = []
for picks in df['blue_picks']:
    champs = ast.literal_eval(picks)  # Convert string to list
    all_picks.extend(champs)

for picks in df['red_picks']:
    champs = ast.literal_eval(picks)
    all_picks.extend(champs)

# Count and show top 10
pick_counts = Counter(all_picks)
for champ, count in pick_counts.most_common(10):
    print(f"{champ:20} {count:3} picks")

print("\n" + "=" * 50)
print("CHAMPION WIN RATES (Min 5 games)")
print("=" * 50)

# Calculate win rates
champ_wins = {}
champ_games = {}

for _, row in df.iterrows():
    blue_champs = ast.literal_eval(row['blue_picks'])
    red_champs = ast.literal_eval(row['red_picks'])
    blue_won = row['blue_win']

    # Count wins for blue team
    for champ in blue_champs:
        champ_games[champ] = champ_games.get(champ, 0) + 1
        if blue_won:
            champ_wins[champ] = champ_wins.get(champ, 0) + 1

    # Count wins for red team
    for champ in red_champs:
        champ_games[champ] = champ_games.get(champ, 0) + 1
        if not blue_won:
            champ_wins[champ] = champ_wins.get(champ, 0) + 1

# Calculate win rates
champ_winrates = {}
for champ in champ_games:
    if champ_games[champ] >= 5:  # At least 5 games
        winrate = champ_wins.get(champ, 0) / champ_games[champ] * 100
        champ_winrates[champ] = winrate

# Show top and bottom 5
sorted_winrates = sorted(champ_winrates.items(), key=lambda x: x[1], reverse=True)

print("\nHighest win rates:")
for champ, wr in sorted_winrates[:5]:
    games = champ_games[champ]
    print(f"{champ:20} {wr:.1f}% ({games} games)")

print("\nLowest win rates:")
for champ, wr in sorted_winrates[-5:]:
    games = champ_games[champ]
    print(f"{champ:20} {wr:.1f}% ({games} games)")

print("\n✅ Data looks good! Ready for modeling.")

Total matches: 99
Blue team wins: 53 (53.5%)
Red team wins: 46 (46.5%)

MOST PICKED CHAMPIONS
Jayce                 30 picks
Ambessa               21 picks
Khazix                20 picks
Sona                  19 picks
Jax                   18 picks
Varus                 18 picks
Ziggs                 17 picks
Nautilus              17 picks
Riven                 17 picks
Yunara                16 picks

CHAMPION WIN RATES (Min 5 games)

Highest win rates:
Neeko                87.5% (8 games)
Vayne                85.7% (7 games)
Rengar               83.3% (6 games)
Riven                82.4% (17 games)
Udyr                 80.0% (5 games)

Lowest win rates:
Lux                  22.2% (9 games)
Jinx                 20.0% (10 games)
Gwen                 20.0% (5 games)
Zyra                 0.0% (5 games)
Fiora                0.0% (7 games)

✅ Data looks good! Ready for modeling.
